In [10]:
import agentpy as ap
import matplotlib.pyplot as plt
import matplotlib.animation
import seaborn as sns
import IPython
import math
import random

class StreetIntersection(ap.Model):
  
  def setup(self):
    # Traffic lights positions in grid
    trafficLightsPositions = [(12,10), (8,10), (10,8), (10,12)]
    
    # Cars set position on axis (for each direction)
    carPositions = [11, 9, 9, 11]

    # Create agents
    self.trafficLights = ap.AgentList(self, 4)
    self.cars = ap.AgentList(self, self.p.cars)

    # Create grid (street intersection)
    self.intersection = ap.Grid(self, (20,20), torus = True, track_empty = True)
    self.intersection.add_agents(self.trafficLights, trafficLightsPositions, empty = False)
    self.intersection.add_agents(self.cars, random = True, empty = True)

    # Initiate dynamic variables for cars
    # moving - 0: false, 1: true
    self.cars.moving = 1
    
    # direction - 0: up, 1: down, 2: left, 3: right
    for car in self.cars:
      carDirection = random.randint(0,3)
      if carDirection == 0:
        car.direction = carDirection
        self.intersection.move_to(car, (random.randint(13,20), carPositions[3]))
      elif carDirection == 1:
        car.direction = carDirection
        self.intersection.move_to(car, (random.randint( 0,7 ), carPositions[2]))
      elif carDirection == 2:
        car.direction = carDirection
        self.intersection.move_to(car, (carPositions[0], random.randint( 0,7 )))
      elif carDirection == 3:
        car.direction = carDirection
        self.intersection.move_to(car, (carPositions[1], random.randint(13,20)))
      
    self.cars.color = 3
    self.cars.id = 1000

    # Initiate a dynamic variable for Traffic lights
    # color - 0: red (stop), 1: yellow (go with precaution), 2: green (go), 3: cars (does not apply for Traffic lights)
    self.trafficLights.color = 1
    self.trafficLights.direction = 10000
    self.trafficLights.cooldown = 0

    i = 0
    # Assign IDs to Traffic lights
    for trafficLight in self.trafficLights:
      trafficLight.id = i
      i += 1

  def step(self):
    minTrafficLightID = 0
    min = 100000
    
    # Find the Traffic light with the closest car
    for trafficLight in self.trafficLights:
      for neighbor in self.intersection.neighbors(trafficLight,2):
        pos1 = self.intersection.positions[neighbor]
        pos2 = self.intersection.positions[trafficLight]
        distance = math.sqrt( (pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2 )
        if(min>distance and neighbor.direction == trafficLight.id):
          min = distance
          minTrafficLightID = trafficLight.id

    # If no cars are found nearby, turn all Traffic lights yellow
    if min == 100000:
      self.trafficLights.color = 1
    else:
      # Turn green the Traffic light with the closest Car
    
      trafficLight = self.trafficLights.select(self.trafficLights.id == minTrafficLightID)
      # Turn Red the remaining Traffic Lights
      self.trafficLights.color = 0
      
      if trafficLight[0].id == 0 or trafficLight[0].id == 1:
        trafficLight = self.trafficLights.select(self.trafficLights.id == 0)
        trafficLight.color = 2
        trafficLight = self.trafficLights.select(self.trafficLights.id == 1)
        trafficLight.color = 2
      else:
        trafficLight = self.trafficLights.select(self.trafficLights.id == 2)
        trafficLight.color = 2
        trafficLight = self.trafficLights.select(self.trafficLights.id == 3)
        trafficLight.color = 2
      
    # Car movement
    # Car won't move if corresponding Traffic light is red
    for car in self.cars.select(self.cars.moving == 1):

      if car.direction == 0: # Up
        
        for neighbor in self.intersection.neighbors(car,2):
          if neighbor.color == 0 and neighbor.id == car.direction:
            car.moving = 1
            break
        else:
          for neighbor in self.intersection.neighbors(car):
            if neighbor.color == 3 and (self.intersection.positions[neighbor][0]-self.intersection.positions[car][0]) == -1 and (self.intersection.positions[neighbor][1]-self.intersection.positions[car][1]) == 0:
              break
          else:
            self.intersection.move_by(car, (-1,0))
                     
      elif car.direction == 1: # Down

        for neighbor in self.intersection.neighbors(car,2):
          if neighbor.color == 0 and neighbor.id == car.direction:
            car.moving = 1
            break
        else:
          for neighbor in self.intersection.neighbors(car):
            if neighbor.color == 3 and (self.intersection.positions[neighbor][0]-self.intersection.positions[car][0]) == 1 and (self.intersection.positions[neighbor][1]-self.intersection.positions[car][1]) == 0:
              break
          else:
            self.intersection.move_by(car, (1,0))

      elif car.direction == 2: # Right

        for neighbor in self.intersection.neighbors(car,2):
          if neighbor.color == 0 and neighbor.id == car.direction:
            car.moving = 1
            break
        else:
          for neighbor in self.intersection.neighbors(car):
            if neighbor.color == 3 and (self.intersection.positions[neighbor][1]-self.intersection.positions[car][1]) == 1 and (self.intersection.positions[neighbor][0]-self.intersection.positions[car][0]) == 0:
              break
          else:
            self.intersection.move_by(car, (0,1))

      elif car.direction == 3: # left

        for neighbor in self.intersection.neighbors(car,2):
          if neighbor.color == 0 and neighbor.id == car.direction:
            car.moving = 1
            break
        else:
          for neighbor in self.intersection.neighbors(car):
            if neighbor.color == 3 and (self.intersection.positions[neighbor][1]-self.intersection.positions[car][1]) == -1 and (self.intersection.positions[neighbor][0]-self.intersection.positions[car][0]) == 0:
              break
          else:
            self.intersection.move_by(car, (0,-1))
              
# Define parameters
parameters = {
    'cars': 15,
    'steps': 100,
}

# Create single-run animation with custom colors
def animation_plot(model, ax):
  attr_grid = model.intersection.attr_grid('color')
  color_dict = {0:'#FF0000', 1:'#FFFF00', 2:'#00FF00', 3:'#000000', None:'#FFFFFF'}
  ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
  ax.set_title(f"Simulation of a Street intersection\n"
                 f"Time-step: {model.t}, # of cars: "
                 f"{len(model.cars)}")
fig, ax = plt.subplots()
model = StreetIntersection(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))